In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import numpy as np
import sympy as sp
# from scipy.linalg import solve_discrete_are
import matplotlib.pyplot as plt

# internal imports
from mld_dynamics_complementarity import mld, h_nom, simulate
from warm_start_hmpc.controller import HybridModelPredictiveController
from warm_start_hmpc.mcais import mcais, solve_dare
from visualizer import vis, animate

Matrix([[qc + 0.05*vc], [qp + 0.075*vc - 0.025*vp], [vc], [1.5*vc - 0.5*vp]])
Matrix([[qc + 0.05*vc], [qp + 0.075*vc - 0.025*vp], [vc], [1.5*vc - 0.5*vp]])
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [ ]:
# controller horizon
T = 20

# weight matrices
h = h_nom
C = np.eye(mld.nx) * h
D = np.vstack([1.]+[0.]*(mld.nu - 1)).T * h

# # terminal cost
# Bu = mld.B[:,0:1]
# Du = D[:,0:1]
# P = solve_discrete_are(mld.A, Bu, C.T.dot(C), Du.T.dot(Du))
# C_T = np.linalg.cholesky(P).T
# C_T = C
# objective = [C, D, C_T]

# # terminal constraints
# F_T = np.vstack((np.eye(mld.nx), - np.eye(mld.nx)))
# h_T = np.concatenate((x_max, x_max))
# terminal_set = [F_T, h_T]

In [ ]:
from pympc.dynamics.discrete_time_systems import LinearSystem
from pympc.geometry.polyhedron import Polyhedron

# LQR cost to go
Bu = mld.B[:,0:1]
Du = D[:,0:1]
P, K = solve_dare(mld.A, Bu, C.dot(C), Du.dot(Du))
C_T = np.linalg.cholesky(P).T
objective = [C, D, C_T]

# # mcais terminal set
# domain = Polyhedron(
#     np.hstack((mld.F, mld.G[:,0:1])),
#     mld.h
# )
# X_T = S.mcais(K, domain, verbose=True)
# terminal_set = [X_T.A, X_T.b]

In [ ]:
A_cl = mld.A + Bu.dot(K)
lhs = mld.F[:-2] + mld.G[:-2,0:1].dot(K)
rhs = mld.h[:-2]
terminal_set = mcais(A_cl, lhs, rhs, verbose=True)

In [ ]:
# hybrid controller
controller = HybridModelPredictiveController(mld, T, objective, terminal_set)

# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])

## Open-loop solution

In [ ]:
# solve in open loop
solution, leaves = controller.feedforward(
    x0,
    draw_label='Cart pole with walls'
)

In [ ]:
x_gurobi, obj_gurobi = controller.feedforward_gurobi(x0)

In [ ]:
vis.jupyter_cell()

In [ ]:
animate(solution.variables['x'], h*10)

## Solve with and without warm start

In [ ]:
e0 = np.random.randn(mld.nx) * 0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
warm_start = controller.construct_warm_start(leaves, x0, uc0, ub0, e0)

In [ ]:
u0 = np.concatenate((uc0, ub0))
x1 = mld.A.dot(x0) + mld.B.dot(u0) + e0
solution, leaves = controller.feedforward(
    x1,
    warm_start=warm_start,
#     draw_label='Cart pole with walls warm start'
)

In [ ]:
solution, leaves = controller.feedforward(x1)

## Solve in closed loop

In [ ]:
from visualizer import vis, visualize
vis.jupyter_cell()

In [ ]:
# np.random.seed(1)

# set up simulation
T_sim = 30
x_sim = [x0]
u_sim = []
warm_start = None

for t in range(T_sim):
    print('Time step %d.'%t)
    
    # solve miqp
    solution, leaves = controller.feedforward(
        x_sim[-1],
        warm_start=warm_start,
#         printing_period=None
    )
    
    # reorganize solution
    uc0 = solution.variables['uc'][0]
    ub0 = solution.variables['ub'][0]
    u0 = np.concatenate((uc0, ub0))
    x1 = simulate(x_sim[-1], h, uc0[0])[-1]
#     x1 = solution.variables['x'][1]
    e0 = x1 - mld.A.dot(x_sim[-1]) - mld.B.dot(u0)
    print('predicted state:', solution.variables['x'][1])
    print('true state:', x1)
    print('modeling error:', e0)
    visualize(x1)
    
    # generate warm start
    warm_start = controller.construct_warm_start(leaves, x_sim[-1], uc0, ub0, e0)
    
    # retrieve closed-loop trajectory
    x_sim.append(x1)
    u_sim.append(uc0)

## Animation

In [ ]:
from visualizer import vis, animate
vis.jupyter_cell()

In [ ]:
x0 = np.array([0., 0., 1., 0.])
x_sim, h = simulate(x0, 5.)
animate(x_sim, h)